# Una red neuronal convolucional para detectar objetos

In [2]:
import os
import sys
import tarfile
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
from six.moves import urllib
session = tf.Session()

In [3]:
batch_size = 128
output_every = 50
generations = 20000
eval_every = 500
image_height = 32
image_width = 32
crop_height = 24
crop_width = 24
num_channels = 3
num_targets = 10
data_folder = "cifar-10-batches-bin"

In [4]:
learning_rate = 0.1
lr_decay = 0.9
num_gens_to_wait = 250

In [5]:
image_vect_length = image_height*image_width*num_channels
record_length = 1+image_vect_length

# Descarga y procesamiento de CIFAR 10

In [7]:
data_dir = 'cifar-10-temp'
if not os.path.exists(data_dir):
    os.makedirs(data_dir)
cifar10_url = "http://www.cs.toronto.edu/~kriz/cifar-10-binary.tar.gz"
data_file = os.path.join(data_dir, 'cifar-10-binary.tar.gz')
if not os.path.isfile(data_file):
    filepath, _ = urllib.request.urlretrieve(cifar10_url, data_file)
    tarfile.open(filepath, 'r:gz').extractall(data_dir)

In [11]:
def read_cifar_files(filename_queue, distort_images = True):
    reader = tf.FixedLengthRecordReader(record_bytes=record_length)
    key, record_string = reader.read(filename_queue)
    #creamos fichero binario
    record_bytes = tf.decode_raw(record_string, tf.uint8)
    #extraemos la etiqueta
    image_label = tf.cast(tf.slice(record_bytes, [0], [1]), tf.int32)
    #extraemos la imagen
    image_extracted = tf.reshape(tf.slice(record_bytes, [1], [image_vect_length]), [num_channels, image_height, image_width])
    #redimensión de imagen
    reshaped_image = tf.transpose(image_extracted, [1,2,0])
    reshaped_image = tf.cast(reshaped_image, tf.float32)
    #crop aleatorio
    final_image = tf.image.resize_image_with_crop_or_pad(reshaped_image, crop_width, crop_height)
    if distort_images:
        #flip aleatorio horizontal, cambiar brillo y contraste
        final_image = tf.image.random_flip_left_right(final_image)
        final_image = tf.image.random_brightness(final_image, max_delta=63)
        final_image = tf.image.random_contrast(final_image, lower=0.2, upper=1.8)
    #estandarización por color
    final_image = tf.image.per_image_standardization(final_image)
    return final_image, image_label

In [12]:
def input_pipeline(batch_size, train_logical=True):
    if train_logical:
        files = [os.path.join(data_dir, data_folder, 'data_batch_{}.bin'.format(i)) for i in range(1,6)]
    else:
        files = [os.path.join(data_dir, data_folder, 'test_batch.bin')]
    
    filename_queue = tf.train.string_input_producer(files)
    image, label = read_cifar_files(filename_queue)
    
    min_after_dequeue = 1000
    capacity = min_after_dequeue+3*batch_size
    example_batch, label_batch = tf.train.shuffle_batch([image,label], batch_size, capacity, min_after_dequeue)
    return example_batch, label_batch